In [1]:
import astropy.units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, ICRS, CartesianRepresentation, CartesianDifferential
from astropy.time import Time
import pandas as pd

In [2]:
f = pd.read_excel('input/alt_az_radial.xlsx', sheet='Sheet1')
f_vel = pd.read_excel('input/asteroid_vel.xlsx', sheet="Sheet1")

In [21]:
loc = EarthLocation(lat = 22.5726*u.deg, lon = 88.3639*u.deg, height = 0*u.m)
time = Time(1515110400, format='unix')

helio_data = []
# pd.DataFrame(columns=['Name of asteroid', 'x', 'y', 'z', 'v_x', 'v_y', 'v_z'])
for i in range(0, 400):
    newAltAz = AltAz(alt = f.iloc[i][2]*u.deg, az = f.iloc[i][1]*u.deg, distance=f.iloc[i][3]*u.km, obstime = time, location = loc)
    newAltAz.representation = "cartesian"
    velocity = AltAz(x = newAltAz.x, 
                     y = newAltAz.y, 
                     z = newAltAz.z, 
                     obstime = time, 
                     location = loc, 
                     v_x=f_vel.iloc[i][1]*u.km/u.s, 
                     v_y=f_vel.iloc[i][2]*u.km/u.s, 
                     v_z=f_vel.iloc[i][3]*u.km/u.s, 
                     representation=CartesianRepresentation, 
                     differential_cls=CartesianDifferential
                    )
    obj = velocity.transform_to(ICRS)
    obj.representation = "cartesian"
    obj.differential_cls = CartesianDifferential
    dict1 = {
        'Name of asteroid' : f.iloc[i][0],
        'x' : obj.x,
        'y' : obj.y,
        'z' : obj.z,
        'v_x' : obj.v_x,
        'v_y' : obj.v_y,
        'v_z' : obj.v_z
    }
    helio_data.append(dict1)
df = pd.DataFrame(helio_data, columns=['Name of asteroid', 'x', 'y', 'z', 'v_x', 'v_y', 'v_z'])

In [20]:
file_name = 'input/helio_data.csv'
df.to_csv(file_name, encoding='utf-8')